# Tabular data manipulation

In the Introduction to Python class, we spent a good deal of time learning to manipulate tabular data with the `pandas` library and create plots with `matplotlib`. In this exercise, we will recap loading, manipulating, and plotting tabular data.

As always, we first need to load the libraries we will be using—just `pandas` and `matplotlib` in this case, and we're giving them the conventional aliases `pd` and `plt` to save typing.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Loading data

As before, we'll use `pandas` `read_csv` function to read a CSV file. The data we have here are data on traffic counts in New York City, from the [New York City open data portal](https://data.cityofnewyork.us/Transportation/Traffic-Volume-Counts/btm5-ppia).

In [ ]:
data = pd.read_csv("../data/Traffic_Volume_Counts.csv")

In [ ]:
data

In general, I'm not a big fan of column names with special characters in them. We can leave the time columns as they are for now, but let's rename the roadway name column.

In [ ]:
data = data.rename(columns={"Roadway Name": "roadway_name"})

## Joins

Oftentimes, the data you want will be in multiple files that you have to join together based on some common identifier. In this case, we'd like to know what borough of New York City each of these segments are in. That is in the `../data/Traffic_Borough.csv` file, extracted from the [New York City LION geospatial database](https://www.nyc.gov/site/planning/data-maps/open-data/dwn-lion.page).

In [ ]:
borough = pd.read_csv("../data/Traffic_Borough.csv")
borough

What column would we use to link this data to the original dataset?

The `RBoro` column contains numeric codes for the five boroughs, but for our analysis, we want strings.

The codes are 1 for Manhattan, 2 for the Bronx, 3 for Brooklyn, 4 for Queens, and 5 for Staten Island. We can use the `.recode` function to create a new `borough` field in the dataset.

In [ ]:
borough["borough"] = borough.RBoro.replace({
    1: "Manhattan",
    2: "Bronx",
    3: "Brooklyn",
    4: "Queens",
    5: "Staten Island"
}).astype("category")


Next, we can join the data with the borough file. We use the `.merge` function to do this. Recall that there are left, right, inner, and outer joins; in this case we'll specify we want a left join. I always recommend validating your joins as well, to make sure the data match the way you expect them to. In this case, this should be a many-to-one join: there should be multiple records per segment in the traffic data, and only a single record per segment in the boroughs file.

In [ ]:
data = data.merge(borough, on="SegmentID", how="left", validate="m:1")

Well, we got an error - this is why I like to validate my merges. The error message notes that the merge keys are not unique in the right dataset, and therefore this is not a many-to-one join. Let's investigate which ones are duplicated. keep=False tells pandas to show us all duplicated records - usually it would only show us the ones after the first duplicated record.

In [ ]:
borough[borough.SegmentID.duplicated(keep=False)]

Many of these look like they are true duplicates in the data - for instance, segment 77356 is recorded four times, but they all consistently say this segment is in the Bronx. We can use the `.drop_duplicates` function to drop the true duplicate rows in the borough dataframe.

In [ ]:
borough = borough.drop_duplicates()
data = data.merge(borough, on="SegmentID", how="left", validate="m:1")

We still get the same error. What does that mean?

That must mean there were some duplicated segment IDs that _didn't_ have the same borough. Let's take a look at the duplicates again.

In [ ]:
borough[borough.SegmentID.duplicated(keep=False)]

I don't see any duplicated segment IDs in the preview, which must mean the duplicates are not in order. Let's look at a specific case, segment 69424.

In [ ]:
borough.loc[borough.SegmentID == 69424]

Ah, one record has NaN values. We don't have any use for NaN borough names, so we can drop those. Hopefully this will solve our duplicates problem

In [ ]:
borough = borough.dropna(subset="borough")
borough.loc[borough.SegmentID.duplicated(keep=False)]

Good, there are no more duplicate segment IDs. Now, let's just add an assertion that that is the case - this is just an automated check that some assumption we made is true. The check will be run every time we run the notebook, and will cause an error if it ever stops being true (for example, due to a code change above).

There's no need to use keep=False here, if anything is duplicated there will be at least one `True` value regardless of the value of `keep`

In [ ]:
assert not borough.SegmentID.duplicated().any()

Now, we can finally do the merge.

In [ ]:
data = data.merge(borough, on="SegmentID", how="left", validate="m:1")

Now, let's check the quality of the merge, by seeing how many failed to merge, and inspecting those that failed to merge.

In [ ]:
data.borough.isnull().mean()

In [ ]:
data.loc[data.borough.isnull()]

Unfortunately, this is quite a few streets (almost 10%) where the merge failed. If I were doing a more extensive project, I'd investigate this further, but for our purposes we'll just ignore it.

## Descriptive statistics

Many descriptive statistics are available as functions of columns. For instance, we can use `.mean()` to calculate the mean number of vehicles that cross a sensor between 2 and 3 pm. We have to refer to the column using subscript notation (`["column"]`) instead of the usual dot notation `.name` because the column names are not valid Python variable names (they start with numbers and include colons and dashes).

In [ ]:
data["2:00-3:00PM"].mean()

## Grouped data analysis/split-apply-combine

We can use `groupby` to split our dataset into pieces based on the values of some variable, and then apply analyses to each group individually. For instance, we can compute the mean number of vehicles crossing between 2 and 3 PM by sensor and direction. When groupby by multiple variables in `pandas`, you must enclose the group variables in a list (`[]`)

In [ ]:
data.groupby(["SegmentID", "Direction"])["2:00-3:00PM"].mean()

## Data types

Every column in a `pandas` data frame has a data type. We can view these data types with the `.dtypes` property of the data frame.

In [ ]:
data.dtypes

We see that all of the columns with counts are `float64` - or floating point (decimal) number. Looking at the data, you might expect these to be integers (`int64`), but `pandas` has autodetected them as `float64`. This commonly happens with numeric columns with missing data. Floating-point numbers allow `pandas` to represent missing data as `NaN` or not-a-number, which is a special value of floating point number. There is no such value for integers, which would force `pandas` to store the missing values separately, slightly increasing memory consumption.

ID and SegmentID are `int64`, indicating that these are integers (as expected) with no missing values.

Roadway Name, From, To, and Date are stored as `object` columns. These can represent any Python object, but are most often used to represent strings (text). We expect Roadway Name, From, and To to be text. However, Date should be stored as a date column if we want to do any analysis with it (e.g. sorting, selecting specific months, plotting, etc.). There are two ways to do this. The first is to parse the column, using `pd.to_datetime`. Specifying a format is optional, but I always like to do it to make sure that there is no confusion about whether the dates were month-day-year, year-month-day, etc. You can [see the documentation](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes) for what codes you can use in the format description.

I like to print out the result before overwriting the variable, to make sure everything parsed correctly.

In [ ]:
pd.to_datetime(data.Date, format="%m/%d/%Y")

Next, we can overwrite the Date field using the result of our `pd.to_datetime` call. We have to use subscript notation `["column"]` here, because we are creating or overwriting a column.

In [ ]:
data["Date"] = pd.to_datetime(data.Date, format="%m/%d/%Y")

Then, we can check the dtypes again to make sure it worked.

In [ ]:
data.dtypes

## Plotting

Often, plots and graphs are the most effective way to present data. The `matplotlib` library provides extensive functionality for plotting. Here, we will plot a histogram of the dates, and create a line plot of afternoon traffic by day.

In [ ]:
plt.hist(data.Date)

To plot average afternoon traffic by day, we first have to use a groupby to calculate average afternoon traffic, and then we can plot.

In [ ]:
by_day = data.groupby("Date")["4:00-5:00PM"].mean().reset_index()
by_day

In [ ]:
plt.plot(by_day.Date, by_day["4:00-5:00PM"])

Why might this graph be misleading?